In [1]:
!pip install cassandra-driver

In [2]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2 , CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider

In [3]:
cluster_cassandra = Cluster(['127.0.0.1'], port=9042)

In [4]:
session_cassandra = cluster_cassandra.connect()

In [5]:
command = """
CREATE KEYSPACE IF NOT EXISTS badariks

WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 2};
"""

In [6]:
session_cassandra.execute(command)

In [7]:
command = """
CREATE TABLE IF NOT EXISTS badariks.badari_table (
   no int PRIMARY KEY,
   name text)
"""

In [8]:
session_cassandra.execute(command)

In [13]:
command = """
USE badariks
"""

In [14]:
session_cassandra.execute(command)

In [15]:
command = """
insert into badariks.badari_table (no, name) VALUES(1, 'badari')
"""

In [16]:
session_cassandra.execute(command)

In [17]:
!pip install pandas
import pandas as pd

In [27]:
dfemployees = pd.read_csv("badari.csv")

In [28]:
dfemployees

,no,name
0,1,babu
1,2,badari
2,3,narendra
3,4,vaskuri


In [39]:
for emp in dfemployees.iterrows():
    #print(emp[0])
    emp_no = emp[1][0]
    emp_name = emp[1][1]
    print(emp_no, emp_name)
    print("--------------")
    command = f"""
    insert into source.employees (no, name) VALUES({emp_no}, '{emp_name}')
    """
    session_cassandra.execute(command)
    
    
    

1 babu
--------------
2 badari
--------------
3 narendra
--------------
4 vaskuri
--------------


/tmp/ipykernel_6534/3804943730.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  emp_no = emp[1][0]
/tmp/ipykernel_6534/3804943730.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  emp_name = emp[1][1]


In [40]:
r = session_cassandra.execute('select * from badariks.badari_table')
print(r.current_rows)

[Row(no=1, name='badari')]


In [41]:
pip install aws

Note: you may need to restart the kernel to use updated packages.


In [42]:
ssl_context = SSLContext(PROTOCOL_TLSv1_2 )
ssl_context.load_verify_locations('sf-class2-root.crt')

/tmp/ipykernel_6534/972152945.py:1: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2 )


In [43]:
ssl_context.verify_mode = CERT_REQUIRED
auth_provider = PlainTextAuthProvider(username='u1-at-044323040668', password='2/XrmsY7nIBQJCl0KBgZjhWY9P/SYM3DRPZ4w8dopRI=')

In [44]:
cluster_aws = Cluster(['cassandra.us-east-1.amazonaws.com'], ssl_context=ssl_context, auth_provider=auth_provider, port=9142)

In [45]:
session_aws = cluster_aws.connect()

In [46]:
command = """
select * from badariks.badari_table
"""

In [48]:
rows_source_emp = session_cassandra.execute(command)

In [54]:
command = """
CREATE KEYSPACE IF NOT EXISTS babuks

WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 2};
"""

In [56]:
session_aws.execute(command)

In [78]:
command = """
CREATE TABLE IF NOT EXISTS babuks.employee1 (
   no int PRIMARY KEY,
   name text)
"""

In [79]:
session_aws.execute(command)

In [80]:
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement

for row in rows_source_emp:
    query = SimpleStatement(
        "INSERT INTO  babuks.employee1(no, name) VALUES (%s, %s)",
        consistency_level=ConsistencyLevel.LOCAL_QUORUM)
    session_aws.execute(query, row)

In [82]:
row = session_aws.execute("select * from babuks.employee1")

In [83]:
for row in rows:
    print(row)